**Installation**

# Translating medical terminology for rare diseases used by laypeople into the Human Phenotype Ontology with NLP and the BERT model

## BlueBERT implementation for comparison

## Package Installation

Below is the command to install all the required packages for the project.

In [1]:
# Installing necessary packages
!pip install transformers pandas numpy torch scikit-learn keras-preprocessing plotly prettytable

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.4 MB/s eta 0:00:00


## Importing Required Libraries and Packages

Below are the import statements required for the project.

In [2]:
# Importing necessary libraries from the transformers package for BERT model
from transformers import BertTokenizer, BertForSequenceClassification

# Importing data manipulation libraries
import pandas as pd
import numpy as np

# Importing torch for neural network and related operations
import torch
import torch.nn as nn
import torch.nn.functional as F

# Importing DataLoader and TensorDataset for batching and managing datasets in PyTorch
from torch.utils.data import DataLoader, TensorDataset

# Importing loss functions from PyTorch for binary and multi-class classification
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss

# Importing optimization algorithms from PyTorch
from torch.optim import AdamW, SGD

# Importing learning rate scheduler from PyTorch
from torch.optim.lr_scheduler import StepLR

# Importing train-test split function from scikit-learn for splitting datasets
from sklearn.model_selection import train_test_split

# Importing accuracy score from scikit-learn for model evaluation
from sklearn.metrics import accuracy_score

# Importing sequence padding function from Keras preprocessing
from keras_preprocessing.sequence import pad_sequences

# Importing various metrics from scikit-learn for model evaluation
from sklearn.metrics import (
    precision_recall_fscore_support, 
    hamming_loss, 
    jaccard_score, 
    log_loss, 
    accuracy_score, 
    roc_auc_score
)

# Importing Plotly's graph objects for data visualization
import plotly.graph_objects as go

# Importing PrettyTable for tabular data representation
from prettytable import PrettyTable

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Data Loading and Preliminary Analysis

We load a dataset from a specified path, display its dimensions, show a few rows of data, compute and display the maximum sentence length in a specific column, and report the total number of records in the following code snippets. Finally, the text data and labels are separated for further processing.

### Note
If running on local or any other machine, please change the file path below.

In [3]:
# Load the data from a CSV file
# The commented line is an alternative path for a different dataset
# df = pd.read_csv('')
df = pd.read_csv('/kaggle/input/r-diseases-dataset/model_input_latest.csv')

# Print the shape of the dataframe to get an understanding of its size
print(df.shape)

(16935, 243)


In [4]:
# Display the first 5 rows of the data to understand its structure
df.head(5)

,week_description,HP:0000006,HP:0000010,HP:0000012,HP:0000019,HP:0000103,HP:0000107,HP:0000201,HP:0000211,HP:0000217,...,HP:0100651,HP:0100749,HP:0100752,HP:0100759,HP:0100760,HP:0100851,HP:0100852,HP:0100868,HP:0100891,HP:0200099
0,"At the start of this week, I began experiencin...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"At the start of this week, I began experiencin...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The tingling and numbness I mentioned earlier ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The tingling and numbness I mentioned earlier ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,I've started experiencing episodes of dizzines...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# Initialize a variable to keep track of the maximum sentence length
max_sentence_length = 0

# Iterate through each record in the dataset
for index, row in df.iterrows():
    # Extract the week description from the current record
    week_description = row['week_description']
    
    # Compute the length of the sentence by splitting it into words
    sentence_length = len(week_description.split())
    
    # Update the maximum sentence length if the current sentence is longer
    max_sentence_length = max(max_sentence_length, sentence_length)

# Print the maximum sentence length
print(f"Maximum sentence length: {max_sentence_length}")

Maximum sentence length: 89


In [6]:
# Report the total number of records in the dataset
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

# Separate the text data and labels for further processing
# Extract the 'week_description' column as the text data
texts = df['week_description'].values

# Drop the 'week_description' column and use the remaining columns as labels
labels = df.drop('week_description', axis=1).values

Number of training sentences: 16,935



## Text Tokenization and Input Preparation

In this part of the code, we focus on preparing our text data for the BERT model by tokenizing the text, padding and truncating the token sequences, and creating attention masks to indicate which tokens are meaningful and which are padding.

In [7]:
# Instantiate a BlueBERT tokenizer to convert text to token IDs
tokenizer = BertTokenizer.from_pretrained('bionlp/bluebert_pubmed_mimic_uncased_L-12_H-768_A-12')

# Step 1: Tokenize Text
# Tokenize the text data, adding the special tokens [CLS] and [SEP] as required by BERT
input_ids = [tokenizer.encode(text, add_special_tokens=True) for text in texts]

# Step 2: Padding and Truncating
# Ensure that all sequences are of the same length by padding and truncating
# We choose a maximum length of 128 tokens for this purpose
input_ids = pad_sequences(
    input_ids, 
    maxlen=128, 
    dtype="long", 
    value=0,  # Value used for padding
    truncating="post",  # Truncate sequences from the end if necessary
    padding="post"  # Pad sequences at the end if necessary
)

# Step 3: Create Attention Masks
# Create attention masks to differentiate actual tokens from padding
# A mask value of 1 indicates a real token, while a value of 0 indicates padding
attention_masks = []
for seq in input_ids:
    # Create a mask for the current sequence
    seq_mask = [float(i > 0) for i in seq]  # i > 0 checks whether the token ID is not padding (0)
    attention_masks.append(seq_mask)

## Data Splitting and DataLoader Preparation

In this section, we split the input data and labels into training, validation, and test sets to prepare them for model training, validation, and testing. Additionally, we convert the data into PyTorch tensors and organize them into DataLoaders for efficient batch processing during training and evaluation.

In [8]:
# Step 1: Splitting the Data
# Split the input data and labels into training and temporary sets (for further splitting)
train_inputs, temp_inputs, train_labels, temp_labels = train_test_split(
    input_ids, labels, 
    random_state=42,  # Ensures reproducibility
    test_size=0.2  # Specifies the proportion of the data to include in the test set
)

# Split the temporary sets into validation and test sets
val_inputs, test_inputs, val_labels, test_labels = train_test_split(
    temp_inputs, temp_labels, 
    random_state=42,  # Ensures reproducibility
    test_size=0.5  # Specifies the proportion of the data to include in the test set
)

# Also split the attention masks in a similar fashion
train_masks, temp_masks, _, _ = train_test_split(attention_masks, labels, random_state=42, test_size=0.2)
val_masks, test_masks, _, _ = train_test_split(temp_masks, temp_labels, random_state=42, test_size=0.5)

# Step 2: Converting Data to PyTorch Tensors
# Convert all inputs, masks, and labels to PyTorch tensors as required for training in PyTorch
train_inputs = torch.tensor(train_inputs)
val_inputs = torch.tensor(val_inputs)
test_inputs = torch.tensor(test_inputs)

train_masks = torch.tensor(train_masks)
val_masks = torch.tensor(val_masks)
test_masks = torch.tensor(test_masks)

train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)
test_labels = torch.tensor(test_labels)

# Step 3: Creating DataLoaders
# Organize the data into DataLoaders for efficient batch processing during training and evaluation
# Set the batch size to 32
batch_size = 32

# Create DataLoader for the training set
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_dataloader = DataLoader(train_data, batch_size=batch_size)

# Create DataLoader for the validation set
val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_dataloader = DataLoader(val_data, batch_size=batch_size)

# Create DataLoader for the test set
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_dataloader = DataLoader(test_data, batch_size=batch_size)


## Model Initialization and Device Preparation

In this section, we initialize the BERT model for sequence classification and a linear classifier. We also prepare the computing device (CPU or GPU) for training and evaluation.

In [9]:
# Step 1: Initialize BlueBERT Model
model = BertForSequenceClassification.from_pretrained(
    "bionlp/bluebert_pubmed_mimic_uncased_L-12_H-768_A-12", 
    num_labels=train_labels.shape[1]  # The number of output labels equals the number of columns in train_labels
)

# Step 2: Initialize Linear Classifier
# Initialize a linear classifier to be trained alongside the BERT model
# The input and output dimensions are both equal to the number of labels
classifier = torch.nn.Linear(
    train_labels.shape[1], 
    train_labels.shape[1]
)

# Step 3: Prepare Computing Device
# Determine the computing device (CPU or GPU) and send the model and classifier to this device
device = torch.device(
    "cuda" if torch.cuda.is_available() else "cpu"  # Use GPU (cuda) if available, otherwise fall back to CPU
)

# Send the model and classifier to the chosen device
model.to(device)
classifier.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bionlp/bluebert_pubmed_mimic_uncased_L-12_H-768_A-12 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Linear(in_features=242, out_features=242, bias=True)

## Calculating Class Weights

In this section, we calculate the class weights which are useful to handle imbalanced datasets during training.

In [10]:
# Step 1: Count Positive and Negative Samples
# Count the number of positive samples for each label by summing up the training labels along the column axis
n_pos = train_labels.sum(axis=0)
# Uncomment the line below to print the count of positive samples for debugging
# print(n_pos)

# Calculate the number of negative samples for each label by subtracting the count of positive samples from the total count
n_neg = len(train_labels) - n_pos
# Uncomment the line below to print the count of negative samples for debugging
# print(n_neg)

# Step 2: Calculate Positive Class Weights
# The positive class weights are calculated as the ratio of negative samples to positive samples for each label
# Adding a small constant (1e-5) to avoid division by zero
pos_weights = (n_neg + 1e-5) / (n_pos + 1e-5)

# Uncomment the line below to print the calculated positive class weights for debugging
# print(f"Positive Weights: {pos_weights}")

# Step 3: Convert to Tensor and Send to Device
# Convert the numpy array of positive class weights to a PyTorch tensor
# Then send the tensor to the chosen computing device (CPU or GPU)
pos_weights = torch.tensor(pos_weights).to(device)


/tmp/ipykernel_67/1551397999.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pos_weights = torch.tensor(pos_weights).to(device)


## Optimizer, Learning Rate Scheduler, and Loss Function Initialization

In this section, we initialize the optimizer, learning rate scheduler, and the loss function which are essential components for training the neural network.

In [11]:
# Step 1: Initialize Optimizer
# We use the AdamW optimizer which is an extension of Adam optimized for training deep neural networks
# We include the parameters of both the BERT model and the linear classifier in the optimizer
optimizer = AdamW(
    list(model.parameters()) + list(classifier.parameters()),  # Combine the parameters of model and classifier
    lr=0.0001  # Set the learning rate
)

# Step 2: Initialize Learning Rate Scheduler
# The StepLR scheduler adjusts the learning rate at regular intervals for better convergence
scheduler = StepLR(
    optimizer, 
    step_size=10,  # Decrease the learning rate every 10 epochs
    gamma=0.7  # Multiplicative factor to decrease the learning rate
)

# Step 3: Initialize Loss Function
# The pos_weight argument helps handle imbalanced datasets by scaling the loss for positive samples
criterion = BCEWithLogitsLoss(
    pos_weight=pos_weights  # Set the positive weights to handle class imbalance
)

# Alternative Loss Function
# The Focal Loss is another alternative for handling imbalanced datasets
# Uncomment the following lines to use Focal Loss instead of BCEWithLogitsLoss
# criterion = FocalLoss(
#     alpha=1, 
#     gamma=2, 
#     logits=True, 
#     reduce=True
# )


## Utility Functions for Printing Metrics

In this section, we introduce two utility functions for pretty-printing the metrics during training and evaluation.

In [12]:
def pretty_print_metrics(title, metrics):
    """
    Prints the metrics in a tabular format.
    
    :param title: (str) Title to be displayed above the table.
    :param metrics: (dict) Dictionary containing metric names as keys and their values.
    """
    
    # Display the title for the metrics
    print(f"\n{title}")
    
    # Initialize the table
    table = PrettyTable()
    
    # Set the column names for the table
    table.field_names = ["Metric", "Value"]
    
    # Add rows to the table using metrics data
    for metric, value in metrics.items():
        table.add_row([metric, f"{value:.4f}"])
    
    # Print the table
    print(table)

def pretty_print_epoch(epoch, train_loss, val_loss):
    """
    Prints the training and validation loss for each epoch in a tabular format.
    
    :param epoch: (int) Current epoch number.
    :param train_loss: (float) Training loss for the current epoch.
    :param val_loss: (float) Validation loss for the current epoch.
    """
    
    # Display epoch information
    print(f"\n{'='*40}")
    print(f"Epoch {epoch}")
    print(f"{'='*40}")
    
    # Initialize the table
    table = PrettyTable()
    
    # Set the column names for the table
    table.field_names = ["Data Type", "Loss"]
    
    # Add rows to the table for training and validation data
    table.add_row(["Training", f"{train_loss:.4f}"])
    table.add_row(["Validation", f"{val_loss:.4f}"])
    
    # Print the table
    print(table)

## Updating Label Data Types

In this section, we are updating the data types of the label tensors to torch.float32. This data type conversion is essential for ensuring compatibility with PyTorch operations, when using loss functions like BCEWithLogitsLoss.

In [13]:
# Converting the data type of training labels to float32
train_labels = torch.tensor(train_labels, dtype=torch.float32)

# Converting the data type of validation labels to float32
val_labels = torch.tensor(val_labels, dtype=torch.float32)

# Converting the data type of testing labels to float32
test_labels = torch.tensor(test_labels, dtype=torch.float32)

/tmp/ipykernel_67/3817560295.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels = torch.tensor(train_labels, dtype=torch.float32)
/tmp/ipykernel_67/3817560295.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_labels = torch.tensor(val_labels, dtype=torch.float32)
/tmp/ipykernel_67/3817560295.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_labels = torch.tensor(test_labels, dtype=torch.float32)


## Training and Validation Loop

In this section, we set up and run the training and validation loop. We initialize arrays to keep track of training and validation losses over the epochs. In each epoch, we perform a forward and backward pass on the training data, update the model parameters, and compute the average training loss. Then, we evaluate the model on the validation set, compute the average validation loss, and evaluate several performance metrics.

In [14]:
# Arrays to store training and validation loss values across epochs
train_loss_values = []
val_loss_values = []

# Training loop across epochs
for epoch in range(16):
    model.train()  # Set model to training mode
    
    avg_train_loss = 0  # Initialize average training loss for the epoch
    avg_val_loss = 0  # Initialize average validation loss for the epoch
    
    # Loop over batches of training data
    for i, batch in enumerate(train_dataloader):
        
        # Send input data and labels to the device
        inputs, masks, labels = tuple(t.to(device) for t in batch)
        
        # Ensure labels are float for loss computation
        labels = labels.float()

        # Forward pass: compute predictions
        outputs = model(inputs, attention_mask=masks)
        logits_from_model = outputs.logits
        logits = classifier(logits_from_model)
        
        # Compute loss
        loss = criterion(logits, labels)
        
        # Perform backward pass to compute gradients
        loss.backward()
        
        # Gradient clipping to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        # Update model parameters
        optimizer.step()
        optimizer.zero_grad()
        
        # Update average training loss
        avg_train_loss += loss.item() / len(train_dataloader)
        
        # Print training loss every 100 batches
        if i % 100 == 0:
            print(f"Epoch: {epoch}, Batch: {i}, Training Loss: {loss.item()}")
        
    # Store average training loss for the epoch
    train_loss_values.append(avg_train_loss)

    # Validation: Evaluate model on validation data
    model.eval()  # Set model to evaluation mode
    val_preds = []
    val_true = []
    with torch.no_grad():  # No gradient computation
        for batch in val_dataloader:
            
            # Send input data and labels to the device
            inputs, masks, labels = tuple(t.to(device) for t in batch)
            
            # Forward pass: compute predictions
            outputs = model(inputs, attention_mask=masks)
            logits_from_model = outputs.logits
            logits = classifier(logits_from_model)
            
            # Store predictions and true labels for later evaluation
            preds = torch.sigmoid(logits).cpu().numpy()
            val_preds.extend(preds)
            val_true.extend(labels.cpu().numpy())
            
            # Update average validation loss
            avg_val_loss += loss.item() / len(val_dataloader)
        val_loss_values.append(avg_val_loss)

    # Convert prediction probabilities to binary predictions
    val_preds = np.array(val_preds) >= 0.7
    val_true = np.array(val_true)
    
    # Compute validation accuracy
    val_accuracy = accuracy_score(val_true, val_preds)
    
    # Print summary for the epoch
    pretty_print_epoch(epoch, avg_train_loss, avg_val_loss)
    
    # Compute and print several performance metrics on validation data
    precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(val_true, val_preds, average='micro')
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(val_true, val_preds, average='macro')
    hamming = hamming_loss(val_true, val_preds)
    jaccard = jaccard_score(val_true, val_preds, average='samples')
    logloss = log_loss(val_true, val_preds)
    
    val_metrics = {
        'Validation Accuracy': val_accuracy,
        'Micro-average Precision': precision_micro,
        'Micro-average Recall': recall_micro,
        'Micro-average F1': f1_micro,
        'Macro-average Precision': precision_macro,
        'Macro-average Recall': recall_macro,
        'Macro-average F1': f1_macro,
        'Hamming Loss': hamming,
        'Jaccard Similarity': jaccard,
        'Log Loss': logloss
    }

    pretty_print_metrics('Validation Metrics', val_metrics)  # Use PrettyTable to print validation metrics


Epoch: 0, Batch: 0, Training Loss: 1.335249900817871
Epoch: 0, Batch: 100, Training Loss: 1.079715371131897
Epoch: 0, Batch: 200, Training Loss: 0.8810611963272095
Epoch: 0, Batch: 300, Training Loss: 0.5951901078224182
Epoch: 0, Batch: 400, Training Loss: 0.41555601358413696

Epoch 0
+------------+--------+
| Data Type  |  Loss  |
+------------+--------+
|  Training  | 0.8494 |
| Validation | 0.3858 |
+------------+--------+

Validation Metrics
+-------------------------+--------+
|          Metric         | Value  |
+-------------------------+--------+
|   Validation Accuracy   | 0.0130 |
| Micro-average Precision | 0.0904 |
|   Micro-average Recall  | 0.9498 |
|     Micro-average F1    | 0.1650 |
| Macro-average Precision | 0.2057 |
|   Macro-average Recall  | 0.9574 |
|     Macro-average F1    | 0.2952 |
|       Hamming Loss      | 0.0397 |
|    Jaccard Similarity   | 0.1412 |
|         Log Loss        | 3.9597 |
+-------------------------+--------+


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 1, Batch: 0, Training Loss: 0.39615315198898315
Epoch: 1, Batch: 100, Training Loss: 0.25932490825653076
Epoch: 1, Batch: 200, Training Loss: 0.2617379426956177
Epoch: 1, Batch: 300, Training Loss: 0.22337336838245392
Epoch: 1, Batch: 400, Training Loss: 0.18857428431510925

Epoch 1
+------------+--------+
| Data Type  |  Loss  |
+------------+--------+
|  Training  | 0.2685 |
| Validation | 0.1904 |
+------------+--------+

Validation Metrics
+-------------------------+--------+
|          Metric         | Value  |
+-------------------------+--------+
|   Validation Accuracy   | 0.0467 |
| Micro-average Precision | 0.1401 |
|   Micro-average Recall  | 0.9687 |
|     Micro-average F1    | 0.2447 |
| Macro-average Precision | 0.2670 |
|   Macro-average Recall  | 0.9692 |
|     Macro-average F1    | 0.3729 |
|       Hamming Loss      | 0.0247 |
|    Jaccard Similarity   | 0.2266 |
|         Log Loss        | 2.8510 |
+-------------------------+--------+


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 2, Batch: 0, Training Loss: 0.2085455358028412
Epoch: 2, Batch: 100, Training Loss: 0.136262908577919
Epoch: 2, Batch: 200, Training Loss: 0.17005683481693268
Epoch: 2, Batch: 300, Training Loss: 0.13584375381469727
Epoch: 2, Batch: 400, Training Loss: 0.14268891513347626

Epoch 2
+------------+--------+
| Data Type  |  Loss  |
+------------+--------+
|  Training  | 0.1604 |
| Validation | 0.1207 |
+------------+--------+

Validation Metrics
+-------------------------+--------+
|          Metric         | Value  |
+-------------------------+--------+
|   Validation Accuracy   | 0.0851 |
| Micro-average Precision | 0.1749 |
|   Micro-average Recall  | 0.9722 |
|     Micro-average F1    | 0.2964 |
| Macro-average Precision | 0.3300 |
|   Macro-average Recall  | 0.9716 |
|     Macro-average F1    | 0.4412 |
|       Hamming Loss      | 0.0191 |
|    Jaccard Similarity   | 0.2935 |
|         Log Loss        | 2.4768 |
+-------------------------+--------+


...

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Validation Metrics
+-------------------------+--------+
|          Metric         | Value  |
+-------------------------+--------+
|   Validation Accuracy   | 0.1370 |
| Micro-average Precision | 0.2406 |
|   Micro-average Recall  | 0.9669 |
|     Micro-average F1    | 0.3854 |
| Macro-average Precision | 0.4384 |
|   Macro-average Recall  | 0.9684 |
|     Macro-average F1    | 0.5497 |
|       Hamming Loss      | 0.0127 |
|    Jaccard Similarity   | 0.3840 |
|         Log Loss        | 2.3499 |
+-------------------------+--------+
Epoch: 14, Batch: 0, Training Loss: 0.09875725954771042
Epoch: 14, Batch: 100, Training Loss: 0.07911080867052078
Epoch: 14, Batch: 200, Training Loss: 0.06528807431459427
Epoch: 14, Batch: 300, Training Loss: 0.06206272169947624
Epoch: 14, Batch: 400, Training Loss: 0.0853206068277359

Epoch 14
+------------+--------+
| Data Type  |  Loss  |
+------------+--------+
|  Training  | 0.0699 |
| Validation | 0.0543 |
+------------+--------+


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Validation Metrics
+-------------------------+--------+
|          Metric         | Value  |
+-------------------------+--------+
|   Validation Accuracy   | 0.1435 |
| Micro-average Precision | 0.2479 |
|   Micro-average Recall  | 0.9622 |
|     Micro-average F1    | 0.3942 |
| Macro-average Precision | 0.4444 |
|   Macro-average Recall  | 0.9648 |
|     Macro-average F1    | 0.5568 |
|       Hamming Loss      | 0.0122 |
|    Jaccard Similarity   | 0.3898 |
|         Log Loss        | 2.4890 |
+-------------------------+--------+
Epoch: 15, Batch: 0, Training Loss: 0.08084563165903091
Epoch: 15, Batch: 100, Training Loss: 0.060496147722005844
Epoch: 15, Batch: 200, Training Loss: 0.06891963630914688
Epoch: 15, Batch: 300, Training Loss: 0.055781859904527664
Epoch: 15, Batch: 400, Training Loss: 0.0718471109867096

Epoch 15
+------------+--------+
| Data Type  |  Loss  |
+------------+--------+
|  Training  | 0.0664 |
| Validation | 0.0471 |
+------------+--------+

Validation Metrics

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Evaluation on Test Set

This section presents the evaluation of the model on the test set. The model's performance is assessed across various metrics including loss, accuracy, precision, recall, F1-score, Hamming Loss, Jaccard Similarity, and Log Loss. The evaluation is carried out batch-wise, and the predictions along with labels are stored for metric computation.

In [15]:
# Set the model to evaluation mode
model.eval()

# Initialize variables to keep track of total loss and correct predictions
total_loss = 0
total_correct = 0

# Lists to store all predictions and labels
all_preds = []
all_labels = []

# Loop over batches in the test dataloader
for i, batch in enumerate(test_dataloader):
    
    # Unpack the batch and move tensors to the appropriate device
    inputs, masks, labels = tuple(t.to(device) for t in batch)
    
    # Ensure labels are float for loss computation
    labels = labels.float()
    
    # No gradient computation in evaluation phase
    with torch.no_grad():
        # Forward pass: Compute predictions
        outputs = model(inputs, attention_mask=masks)
        logits_from_model = outputs.logits
        logits = classifier(logits_from_model)
        
        # Compute loss
        loss = criterion(logits, labels)
        
        # Accumulate loss
        total_loss += loss.item()
        
        # Convert logits to probabilities and store predictions and labels
        preds = torch.sigmoid(logits).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy())

# Convert list of predictions and labels to NumPy arrays for easier manipulation
all_preds_np = np.array(all_preds)
all_labels_np = np.array(all_labels)

# Binarize predictions based on threshold
binary_preds = all_preds_np >= 0.7

# Calculate Metrics
precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(all_labels_np, binary_preds, average='micro')
precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(all_labels_np, binary_preds, average='macro')
hamming = hamming_loss(all_labels_np, binary_preds)
jaccard = jaccard_score(all_labels_np, binary_preds, average='samples')
logloss = log_loss(all_labels_np, all_preds_np)
accuracy = accuracy_score(all_labels_np, binary_preds)

# Organize metrics in a dictionary and print using pretty_print_metrics function
test_metrics = {
    'Test Loss': total_loss / len(test_dataloader),
    'Test Accuracy': accuracy,
    'Micro-Average Precision': precision_micro,
    'Micro-Average Recall': recall_micro,
    'Micro-Average F1-score': f1_micro,
    'Macro-Average Precision': precision_macro,
    'Macro-Average Recall': recall_macro,
    'Macro-Average F1-score': f1_macro,
    'Hamming Loss': hamming,
    'Jaccard Similarity': jaccard,
    'Log Loss': logloss
}

# Use PrettyTable to display test metrics
pretty_print_metrics('Test Metrics', test_metrics)



Test Metrics
+-------------------------+--------+
|          Metric         | Value  |
+-------------------------+--------+
|        Test Loss        | 0.1247 |
|      Test Accuracy      | 0.1387 |
| Micro-Average Precision | 0.2486 |
|   Micro-Average Recall  | 0.9628 |
|  Micro-Average F1-score | 0.3952 |
| Macro-Average Precision | 0.4579 |
|   Macro-Average Recall  | 0.9745 |
|  Macro-Average F1-score | 0.5660 |
|       Hamming Loss      | 0.0122 |
|    Jaccard Similarity   | 0.3900 |
|         Log Loss        | 1.4640 |
+-------------------------+--------+


## Plotting Training and Validation Loss

In this section, we use Plotly to create a plot that depicts the training and validation loss over epochs. Each point on the plot represents the loss at the end of an epoch, for both training and validation phases. This visualization helps in understanding how the model is learning and whether it's overfitting (if the validation loss starts increasing).

In [16]:
# Create a figure using Plotly's go.Figure()
fig = go.Figure()

# Add a trace for training loss values over epochs
fig.add_trace(
    go.Scatter(
        x=list(range(len(train_loss_values))),  # X-axis: Epoch number
        y=train_loss_values,  # Y-axis: Training loss value
        mode='lines+markers',  # Mode: Lines + Markers for individual data points
        name='Training Loss'  # Trace name
    )
)

# Add a trace for validation loss values over epochs
fig.add_trace(
    go.Scatter(
        x=list(range(len(val_loss_values))),  # X-axis: Epoch number
        y=val_loss_values,  # Y-axis: Validation loss value
        mode='lines+markers',  # Mode: Lines + Markers for individual data points
        name='Validation Loss'  # Trace name
    )
)

# Update the layout of the figure to include title and axis labels
fig.update_layout(
    title='Training and Validation Loss',  # Title of the plot
    xaxis_title='Epoch',  # X-axis label
    yaxis_title='Loss'  # Y-axis label
)

# Display the figure
fig.show()


## Predicting on Sample Texts

In this section, we create a function called predict_sample_texts to preprocess the provided text samples, run them through the model for predictions, and interpret the results to obtain human-readable labels. This function demonstrates how to use a pretrained transformer model and a classifier to predict on new data.

In [17]:
# Function to preprocess and predict on new text data
def predict_sample_texts(texts, model, classifier, tokenizer, device, threshold=0.7):
    results = []  # List to store the results
    
    # Get the column headers except for the first one as a list
    headers = df.columns.tolist()[1:]
    
    for text in texts:
        # Step 1: Preprocess
        # Tokenize the text and add special tokens
        input_id = tokenizer.encode(text, add_special_tokens=True)
        # Pad or truncate the token ids to a fixed length
        input_id = pad_sequences([input_id], maxlen=128, dtype="long", value=0, truncating="post", padding="post")
        
        # Create attention mask
        attention_mask = [[float(i > 0) for i in seq] for seq in input_id]
        
        # Convert to PyTorch tensors and move to the given device
        input_id = torch.tensor(input_id).to(device)
        attention_mask = torch.tensor(attention_mask).to(device)
        
        # Step 2: Model Prediction
        # Set the model and classifier to evaluation mode
        model.eval()
        classifier.eval()
        
        # No gradient computation to save memory
        with torch.no_grad():
            # Obtain model outputs
            output = model(input_id, attention_mask=attention_mask)
            logits_from_model = output.logits
            logits = classifier(logits_from_model)
            # Convert logits to probabilities
            probabilities = torch.sigmoid(logits).cpu().numpy()
            
        # Step 3: Interpret the Result
        # Binarize probabilities based on the threshold
        binary_output = (probabilities >= threshold).astype(int)
        
        # Store the result in a dictionary and append to results list
        results.append({
            'text': text,
            'probabilities': probabilities,
            'binary_output': binary_output,
            # Get the predicted labels
            'predicted_labels': [label for label, output in zip(headers, binary_output[0]) if output == 1]
        })
        
    return results  # Return the results list

# Array of sample texts along with HPO terms for context
# Headache (HP:0002315)
# Dry skin (HP:0000958)
# Nocturia (HP:0000017)
sample_texts = [
    "Still thirsty, still constantly in the bathroom, and now I'm getting headaches that are making it tough to focus. Probably from all the lost fluids. Doctor says my sodium levels are still high. It's like my body's a malfunctioning tap.",
    "I thought things couldn't get worse, but now my skin's as dry as the Sahara. Between this and everything else, I'm feeling pretty miserable.",
    "Been waking up in the middle of the night to use the bathroom. It's affecting my sleep now, and I've started to feel groggy all day long."
]

# Get prediction
results = predict_sample_texts(sample_texts, model, classifier, tokenizer, device)

# Output the results
for i, result in enumerate(results):
    print(f"Sample Text {i+1}: {result['text']}")
    print(f"Predicted Labels: {result['predicted_labels']}")
    print("=" * 50)  # Print a separator


Sample Text 1: Still thirsty, still constantly in the bathroom, and now I'm getting headaches that are making it tough to focus. Probably from all the lost fluids. Doctor says my sodium levels are still high. It's like my body's a malfunctioning tap.
Predicted Labels: ['HP:0000103', 'HP:0000835', 'HP:0000958', 'HP:0002315']
Sample Text 2: I thought things couldn't get worse, but now my skin's as dry as the Sahara. Between this and everything else, I'm feeling pretty miserable.
Predicted Labels: ['HP:0000720', 'HP:0000958', 'HP:0001499']
Sample Text 3: Been waking up in the middle of the night to use the bathroom. It's affecting my sleep now, and I've started to feel groggy all day long.
Predicted Labels: ['HP:0000785']
